In [15]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

bd = pd.read_csv('./compras.csv')

In [18]:
bd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
bd.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
count,10000,10000,7130,6084,4893,3727,2730,1871,1204,729,407,212,106,47,19,6,1,1
unique,105,105,105,105,105,105,105,105,105,104,97,90,62,38,18,6,1,1
top,vinho tinto,queijo,presunto,queijo,presunto,pão,queijo,presunto,pão,pão,presunto,presunto,leite,presunto,tomate,bisnaguinha,arroz,feijão
freq,133,338,225,187,155,120,80,68,36,22,12,8,6,3,2,1,1,1


In [6]:
itens_mais_comprados = bd.stack().value_counts()
itens_mais_comprados.head(20)

queijo               1241
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
salada                837
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
alface                618
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
refrigerante          573
Name: count, dtype: int64

In [17]:
bd = bd.drop(columns=['cliente'])

In [21]:
uniques = bd.stack().unique()
print(uniques)

['limão' 'peito de peru' 'manga' 'cerveja' 'picanha' 'carvão' 'sal'
 'linguiça' 'mel' 'geleia' 'cereal' 'salsicha' 'catchup' 'cebola' 'queijo'
 'frutas' 'chá' 'alface' 'granola' 'feijão' 'farinha de trigo'
 'suco de laranja' 'pimenta' 'sardinha enlatada' 'barra de cereal'
 'molho de tomate' 'gelatina' 'vinho branco' 'pão' 'tempero pronto'
 'vinho tinto' 'presunto' 'arroz' 'detergente' 'filé de frango'
 'papel higiênico' 'salada' 'cenoura' 'maionese' 'molho branco' 'whisky'
 'pudim' 'suco' 'azeitona' 'frango' 'mostarda' 'carne moída' 'nhoque'
 'água de coco' 'laranja' 'cerveja artesanal' 'massa de pizza' 'biscoito'
 'morango' 'bisnaguinha' 'kiwi' 'achocolatado' 'sabão' 'torrada' 'banana'
 'palmito' 'desinfetante' 'sopa pronta' 'ervilha' 'hambúrguer'
 'batata frita' 'refrigerante' 'queijo ralado' 'uva' 'óleo de soja'
 'vodka' 'batata' 'bolo' 'açúcar' 'café' 'leite' 'atum enlatado' 'tomate'
 'manteiga' 'vinagre' 'pepino' 'molho pesto' 'água tônica' 'macarrão'
 'peixe' 'energético' 'castan

In [24]:
transactions = []
for i in range(0,bd.shape[0]):
    transactions.append([str(bd.values[i,j])for j in range(0,bd.shape[1])])

In [37]:
regras = apriori(transactions=transactions, min_support=0.01, min_confidence=0.5, min_lift=10, min_length=2, max_length=2)

In [38]:
resultados = list(regras)
resultados

[RelationRecord(items=frozenset({'açúcar', 'café'}), support=0.0238, ordered_statistics=[OrderedStatistic(items_base=frozenset({'café'}), items_add=frozenset({'açúcar'}), confidence=0.6363636363636364, lift=10.198135198135198)]),
 RelationRecord(items=frozenset({'batata frita', 'hambúrguer'}), support=0.0229, ordered_statistics=[OrderedStatistic(items_base=frozenset({'hambúrguer'}), items_add=frozenset({'batata frita'}), confidence=0.6580459770114943, lift=10.967432950191572)]),
 RelationRecord(items=frozenset({'sorvete', 'chocolate'}), support=0.0241, ordered_statistics=[OrderedStatistic(items_base=frozenset({'chocolate'}), items_add=frozenset({'sorvete'}), confidence=0.6147959183673469, lift=10.145147167778003)]),
 RelationRecord(items=frozenset({'refrigerante', 'pizza'}), support=0.0243, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pizza'}), items_add=frozenset({'refrigerante'}), confidence=0.6214833759590792, lift=11.058423059770092)])]

In [39]:
bd_compras_lista = bd.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(bd_compras_lista).transform(bd_compras_lista)

bd_onehot = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(bd_onehot, min_support=0.003,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

In [40]:
rules.size

14820

In [41]:
rules = rules.sort_values(by='lift', ascending=False)

In [42]:
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
706,"(catchup, hambúrguer)","(cerveja, batata frita, mostarda)",0.0145,0.0056,0.0041,0.282759,50.492611,0.004019,1.386423,0.994617
703,"(cerveja, batata frita, mostarda)","(catchup, hambúrguer)",0.0056,0.0145,0.0041,0.732143,50.492611,0.004019,3.679200,0.985715
709,"(mostarda, hambúrguer)","(cerveja, catchup, batata frita)",0.0110,0.0074,0.0041,0.372727,50.368550,0.004019,1.582406,0.991048
700,"(cerveja, catchup, batata frita)","(mostarda, hambúrguer)",0.0074,0.0110,0.0041,0.554054,50.368550,0.004019,2.217758,0.987453
1066,"(azeite, salada)","(vinagre, tomate, cenoura)",0.0151,0.0054,0.0040,0.264901,49.055678,0.003918,1.353014,0.994634
1051,"(vinagre, tomate, cenoura)","(azeite, salada)",0.0054,0.0151,0.0040,0.740741,49.055678,0.003918,3.798900,0.984934
1277,"(tomate, azeite)","(vinagre, alface, salada)",0.0111,0.0078,0.0042,0.378378,48.510049,0.004113,1.596148,0.990379
1264,"(vinagre, alface, salada)","(tomate, azeite)",0.0078,0.0111,0.0042,0.538462,48.510049,0.004113,2.142617,0.987085
1050,"(vinagre, cenoura, salada)","(tomate, azeite)",0.0076,0.0111,0.0040,0.526316,47.415837,0.003916,2.087678,0.986407
1067,"(tomate, azeite)","(vinagre, cenoura, salada)",0.0111,0.0076,0.0040,0.360360,47.415837,0.003916,1.551499,0.989898
